In [1]:
# imports
import os

os.chdir("/home/tim/Development/OCPPM/")
import pm4py
import pm4py.read
import ocpa.objects.log.importer.ocel.factory as ocel_import_factory
import json
from typing import Any
from copy import copy
import pandas as pd

ocel_in_file = "data/CS/source/cs_january_ots[krs_krv_cv]_oas[oa1_oa2_oa3_oa4_oa5_oa6_oa7_oa8_oa9_oa10_oa11_oa12_oa13_oa15_oa16]_eas[resourceCE_resourceMulti_ea1_ea2_ea3_ea4_ea6_ea8_ea10_ea12_ea14].jsonocel"

In [2]:
ocel = pm4py.read.read_ocel(ocel_in_file)

In [3]:
print(ocel.objects.shape)
ocel.objects.head()

(324035, 17)


,ocel:oid,ocel:type,oa1,oa5,oa2,oa3,oa4,oa6,oa7,oa8,oa9,oa10,oa11,oa12,oa13,oa15,oa16
0,KRS-6830105,krs,0.368881,-0.741550,0.412434,0.442593,0.91578,0.975004,0.942736,0.99729,0.835691,0.255810,0.432954,0.108302,0.865397,0.793746,0.835605
1,KRS-2447124,krs,0.368881,0.821745,0.412434,0.442593,0.91578,0.975004,0.942736,0.99729,0.835691,0.043890,0.562810,0.314775,0.865397,0.793746,0.835605
2,KRV-7509670,krv,1.867732,-1.002100,0.310090,0.290392,0.91578,0.975004,0.942736,0.99729,0.835691,0.255810,0.432954,0.314775,0.865397,0.075241,0.835605
3,KRS-5616964,krs,0.368881,1.082294,0.412434,0.442593,0.91578,0.975004,0.942736,0.99729,0.835691,0.436246,0.562810,0.527404,0.865397,0.793746,0.835605
4,KRV-3723804,krv,0.368881,-1.002100,0.412434,0.290392,0.91578,0.975004,0.942736,0.99729,0.835691,0.091378,0.432954,0.314775,0.865397,0.075241,0.835605


In [4]:
print(ocel.events.shape)
ocel.events.head()

(1752998, 14)


,ocel:eid,ocel:timestamp,ocel:activity,event_ea10,event_ea8,event_ea2,event_ea12,event_ea4,event_ea6,event_resourceCE,event_ea3,event_ea14,event_resourceMulti,event_ea1
0,1,2023-01-01 10:06:44.609,Finish Task,0.224765,0.216395,0.526823,0.274337,4.0,0.26262,0.825684,0.01904,0.019645,0,0.198374
1,2,2023-01-01 10:08:48.993,Finish Task,0.224765,0.216395,0.526823,0.274337,4.0,0.26262,0.825684,0.01904,0.019645,0,0.198374
2,3,2023-01-01 10:08:49.088,Update Object State,0.224765,0.216395,0.526823,0.274337,4.0,0.26262,0.825684,0.01904,0.019645,0,0.198374
3,4,2023-01-01 10:08:49.091,Reallocate Task,0.224765,0.216395,0.526823,0.274337,4.0,0.26262,0.825684,0.01904,0.019645,1,0.198374
4,5,2023-01-01 10:09:46.678,Finish Task,0.040257,0.247333,-0.061526,0.274337,3.0,0.26262,0.825684,0.07390,0.019645,0,0.198374


In [5]:
with open(ocel_in_file, 'r') as ocel:
    data = json.load(ocel)

In [29]:
res = []
for e in data['ocel:events'].values():
    res.extend(e['ocel:omap'])

In [30]:
failure = "KRV-3723804"
res.count(failure)

2